In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["Date"]= dt.datetime.today()

In [6]:
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,Date
0,USA,226978,11975.0,5343.0,+241,10265.0,211370,5403.0,686.0,16.0,2020-04-02 11:59:53.120208
1,Italy,115242,4668.0,13915.0,+760,18278.0,83049,4053.0,1906.0,230.0,2020-04-02 11:59:53.120208
2,Spain,110238,6120.0,10003.0,+616,26743.0,73492,6092.0,2358.0,214.0,2020-04-02 11:59:53.120208
3,Germany,81728,3747.0,997.0,+66,19175.0,61556,3936.0,975.0,12.0,2020-04-02 11:59:53.120208
4,France,56989,0.0,4032.0,0,10935.0,42022,6017.0,873.0,62.0,2020-04-02 11:59:53.120208
...,...,...,...,...,...,...,...,...,...,...,...
200,St. Vincent Grenadines,2,0.0,0.0,0,1.0,1,0.0,18.0,0.0,2020-04-02 11:59:53.120208
201,Sierra Leone,2,0.0,0.0,0,0.0,2,0.0,0.3,0.0,2020-04-02 11:59:53.120208
202,Papua New Guinea,1,0.0,0.0,0,0.0,1,0.0,0.1,0.0,2020-04-02 11:59:53.120208
203,Timor-Leste,1,0.0,0.0,0,0.0,1,0.0,0.8,0.0,2020-04-02 11:59:53.120208


In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Deaths/1M pop,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
1,Italy,115242,4668.0,13915.0,+760,18278.0,83049,4053.0,1906.0,230.0,2020-04-02 11:59:53.120208,12.070013,42.796626,32193.0,0.432237,0.120746,0.125957,0.015209,Last 8 columns are added by Roaya
2,Spain,110238,6120.0,10003.0,+616,26743.0,73492,6092.0,2358.0,214.0,2020-04-02 11:59:53.120208,-3.647550,40.244487,36746.0,0.272220,0.090740,0.143770,0.013046,Last 8 columns are added by Roaya
175,China,81589,35.0,3318.0,+6,76408.0,1863,429.0,57.0,2.0,2020-04-02 11:59:53.120208,103.819074,36.561765,79726.0,0.041618,0.040667,0.311932,0.012685,Last 8 columns are added by Roaya
5,Iran,50468,2875.0,3160.0,+124,16711.0,30597,3956.0,601.0,38.0,2020-04-02 11:59:53.120208,54.274070,32.575033,19871.0,0.159026,0.062614,0.077746,0.004868,Last 8 columns are added by Roaya
4,France,56989,0.0,4032.0,0,10935.0,42022,6017.0,873.0,62.0,2020-04-02 11:59:53.120208,2.618787,47.824905,14967.0,0.269393,0.070750,0.058559,0.004143,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Benin,13,0.0,0.0,0,1.0,12,0.0,1.0,0.0,2020-04-02 11:59:53.120208,2.327853,9.641760,1.0,0.000000,0.000000,0.000004,0.000000,Last 8 columns are added by Roaya
146,Saint Lucia,13,0.0,0.0,0,1.0,12,0.0,71.0,0.0,2020-04-02 11:59:53.120208,-60.969699,13.894795,1.0,0.000000,0.000000,0.000004,0.000000,Last 8 columns are added by Roaya
120,Barbados,45,0.0,0.0,0,0.0,45,0.0,157.0,0.0,2020-04-02 11:59:53.120208,-59.559797,13.181454,0.0,0.000000,0.000000,0.000000,0.000000,Last 8 columns are added by Roaya
148,Greenland,10,0.0,0.0,0,2.0,8,0.0,176.0,0.0,2020-04-02 11:59:53.120208,-41.341911,74.710513,2.0,0.000000,0.000000,0.000008,0.000000,Last 8 columns are added by Roaya


In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"Country":"USA","TotalCases":226978,"NewCases":11975.0,"TotalDeaths":5343.0,"NewDeaths":"+241","TotalRecovered":10265.0,"ActiveCases":211370,"CriticalCases":5403.0,"Tot\\u00a0Cases\\/1M pop":686.0,"Deaths\\/1M pop":16.0,"Date":1585828793120,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":15608.0,"ClosedDeathRate":0.342324449,"CurrentDeathRate":0.0235397263,"Weight":0.0610670298,"DeathRateXWeight":0.0014375012,"notes":"Last 8 columns are added by Roaya"},{"Country":"Italy","TotalCases":115242,"NewCases":4668.0,"TotalDeaths":13915.0,"NewDeaths":"+760","TotalRecovered":18278.0,"ActiveCases":83049,"CriticalCases":4053.0,"Tot\\u00a0Cases\\/1M pop":1906.0,"Deaths\\/1M pop":230.0,"Date":1585828793120,"Longitude":12.07001339,"Latitude":42.79662641,"ClosedCases":32193.0,"ClosedDeathRate":0.4322368217,"CurrentDeathRate":0.1207459086,"Weight":0.1259566177,"DeathRateXWeight":0.0152087462,"notes":"Last 8 columns are added by Roaya"},{"Country":"Spain","TotalCases":110238,"NewCases":612